# Implémentation de DST80

Auteur: [Kaci Amaouche](mailto:amaouchekaci28@gmail.com)

Dans ce [Jupyter](https://jupyter.org/) notebook,nous présentons une implémentation de l'algorithme de chiffrement DST80. Si vous n'êtes pas familier avec Jupyter, vous pouvez jeter un coup d'œil rapide à la documentation ou aux tutoriels disponibles[Notebook Basics](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html) guide (~5min).

Ce notebook comporte:

* [Configuration de l'environnement]
* [Implémentation]

## 1 - Configuration de l'environnement

Pour pouvoir exécuter ces scripts, il va falloir:
1. Ce notebook (the `.ipynb` file)
1. Python >= 3.8
1. Seules les librairies standard sont utilisées 

## 2 - Implémentation

On importe les librairies nécessaires

In [1]:
import binascii
import math, sys

On définit quelques fonctions nécessaires pour l'algorithme de chiffrement

In [13]:
def bit(x, n):
    """
    Récupère le n-ième bit de x.
    
    Args:
    x (int): Le nombre entier dont vous voulez extraire le bit.
    n (int): La position du bit à extraire.
    
    Returns:
    int: La valeur du n-ième bit dans x (soit 0 ou 1).
    """
    return (x >> n) & 1

def bit_slice(x, msb, lsb):
    """
    Extrait les bits de x, en prenant en compte les positions de msb à lsb (incluses).
    
    Args:
    x (int): Le nombre entier dont vous voulez extraire un morceau.
    msb (int): Le bit de poids le plus élevé à inclure dans l'extraction.
    lsb (int): Le bit de poids le plus faible à inclure dans l'extraction.
    
    Returns:
    int: La valeur extraite de x, délimitée par les positions msb et lsb.
    """
    mask = (2 << msb) - 1
    extracted_bits = (x & mask) >> lsb
    return extracted_bits

def bv2i(*args):
    """
    Convertit une séquence de bits en un entier.
    
    Args:
    *args: Un nombre variable d'arguments représentant les bits (0 ou 1).
    
    Returns:
    int: La valeur entière obtenue à partir de la séquence de bits fournie.
    """
    o = 0
    for i in args:
        o = (o << 1) | i
    return o

def fa(x):
    """
    Applique la fonction bit à x en utilisant un masque spécifique.
    
    Args:
    x (int): L'entrée sur laquelle appliquer la fonction bit.
    
    Returns:
    int: La valeur du bit dans x selon le masque 0x3a35acc5.
    """
    return bit(0x3a35acc5, x)

def fb(x):
    """
    Applique la fonction bit à x en utilisant un masque spécifique.
    
    Args:
    x (int): L'entrée sur laquelle appliquer la fonction bit.
    
    Returns:
    int: La valeur du bit dans x selon le masque 0xac35742e.
    """
    return bit(0xac35742e, x)

def fc(x):
    """
    Applique la fonction bit à x en utilisant un masque spécifique.
    
    Args:
    x (int): L'entrée sur laquelle appliquer la fonction bit.
    
    Returns:
    int: La valeur du bit dans x selon le masque 0xb81d8bd1.
    """
    return bit(0xb81d8bd1, x)

def fd(x):
    """
    Applique la fonction bit à x en utilisant un masque spécifique.
    
    Args:
    x (int): L'entrée sur laquelle appliquer la fonction bit.
    
    Returns:
    int: La valeur du bit dans x selon le masque 0x5acc335a.
    """
    return bit(0x5acc335a, x)

def fe(x):
    """
    Applique la fonction bit à x en utilisant un masque spécifique.
    
    Args:
    x (int): L'entrée sur laquelle appliquer la fonction bit.
    
    Returns:
    int: La valeur du bit dans x selon le masque 0xe247.
    """
    return bit(0xe247, x)

def fg(x):
    """
    Applique la fonction bit à x en utilisant un masque spécifique.
    
    Args:
    x (int): L'entrée sur laquelle appliquer la fonction bit.
    
    Returns:
    int: La valeur du bit dans x selon le masque 0x4e72.
    """
    return bit(0x4e72, x)

def h(x):
    """
    Applique un masque et un décalage pour extraire des bits spécifiques de x.
    
    Args:
    x (int): L'entrée à partir de laquelle extraire les bits.
    
    Returns:
    int: La valeur extraite de x en appliquant le masque 0x1f9826f4 et le décalage.
    """
    return (0x1f9826f4 >> (2 * x)) & 3

def fn(s, k):
    """
    Génère un tuple en appliquant différentes fonctions bit aux valeurs de s et k.
    
    Args:
    s (int): Première entrée pour les fonctions bit.
    k (int): Deuxième entrée pour les fonctions bit.
    
    Returns:
    tuple: Un tuple contenant les valeurs résultantes après l'application des fonctions.
    """
    return (
        fd(bv2i(bit(s, 32), bit(k, 32), bit(s, 24), bit(k, 24), bit(s, 16))),
        fe(bv2i(bit(k, 16), bit(s, 8), bit(k, 8), bit(k, 0))),
        fb(bv2i(bit(s, 33), bit(k, 33), bit(s, 25), bit(k, 25), bit(s, 17))),
        fe(bv2i(bit(k, 17), bit(s, 9), bit(k, 9), bit(k, 1))),
        fd(bv2i(bit(s, 34), bit(k, 34), bit(s, 26), bit(k, 26), bit(s, 18))),
        fc(bv2i(bit(k, 18), bit(s, 10), bit(k, 10), bit(s, 2), bit(k, 2))),
        fb(bv2i(bit(s, 35), bit(k, 35), bit(s, 27), bit(k, 27), bit(s, 19))),
        fa(bv2i(bit(k, 19), bit(s, 11), bit(k, 11), bit(s, 3), bit(k, 3))),
        fd(bv2i(bit(s, 36), bit(k, 36), bit(s, 28), bit(k, 28), bit(s, 20))),
        fc(bv2i(bit(k, 20), bit(s, 12), bit(k, 12), bit(s, 4), bit(k, 4))),
        fb(bv2i(bit(s, 37), bit(k, 37), bit(s, 29), bit(k, 29), bit(s, 21))),
        fa(bv2i(bit(k, 21), bit(s, 13), bit(k, 13), bit(s, 5), bit(k, 5))),
        fd(bv2i(bit(s, 38), bit(k, 38), bit(s, 30), bit(k, 30), bit(s, 22))),
        fc(bv2i(bit(k, 22), bit(s, 14), bit(k, 14), bit(s, 6), bit(k, 6))),
        fb(bv2i(bit(s, 39), bit(k, 39), bit(s, 31), bit(k, 31), bit(s, 23))),
        fa(bv2i(bit(k, 23), bit(s, 15), bit(k, 15), bit(s, 7), bit(k, 7)))
    )[::-1]

def g(s, k):
    """
    Génère un tuple en appliquant la fonction fn à s et k, puis applique fg sur les segments du tuple.
    
    Args:
    s (int): Première entrée pour les fonctions fn et fg.
    k (int): Deuxième entrée pour les fonctions fn et fg.
    
    Returns:
    tuple: Un tuple contenant les valeurs résultantes après l'application des fonctions fn et fg.
    """
    fx = fn(s, k)
    return (
        fg(bv2i(*fx[:4])),
        fg(bv2i(*fx[4:8])),
        fg(bv2i(*fx[8:12])),
        fg(bv2i(*fx[12:]))
    )[::-1]

def f(k, s):
    """
    Applique la fonction h sur le résultat obtenu après l'application des fonctions g.
    
    Args:
    k (int): Entrée pour la fonction g.
    s (int): Entrée pour la fonction g.
    
    Returns:
    int: Le résultat de la fonction h après avoir appliqué g.
    """
    return h(bv2i(*g(s, k)))

def p1(x):
    """
    Applique des opérations de masquage et de décalage pour modifier certains bits de x.
    
    Args:
    x (int): L'entrée à modifier.
    
    Returns:
    int: La valeur de x après avoir appliqué les opérations.
    """
    out = x & 0b10100101
    out |= bit(x, 6) << 3
    out |= bit(x, 4) << 1
    out |= bit(x, 3) << 6
    out |= bit(x, 1) << 4
    return out

def p2(x):
    """
    Applique p1 sur des segments de bits de x et les combine pour former la sortie finale.
    
    Args:
    x (int): L'entrée à traiter.
    
    Returns:
    int: La valeur résultante après avoir appliqué p1 sur des segments de bits.
    """
    out = 0
    for i in range(0, 40, 8):
        byte = bit_slice(x, i + 8, i)
        out |= (p1(byte) << i)
    return out


Ensuite, on définit la fonction de génération de clé et la fonction de cycle DST80

In [14]:
def dst80_merge(keyl, keyr):
    """
    Fusionne deux parties de clé en appliquant p2 et effectue des opérations de masquage.
    
    Args:
    keyl (int): Partie gauche de la clé.
    keyr (int): Partie droite de la clé.
    
    Returns:
    int: La clé fusionnée après l'application de p2 et des opérations de masquage.
    """
    keyl = p2(keyl)  # Applique p2 à la partie gauche de la clé
    keyr = p2(keyr)  # Applique p2 à la partie droite de la clé
    
    if bit(keyl, 39) == 1:
        keyl = 0x7fffffffff ^ keyl  # Effectue un XOR avec le masque si le 39e bit est 1
    if bit(keyr, 39) == 1:
        keyr = 0x7fffffffff ^ keyr  # Effectue un XOR avec le masque si le 39e bit est 1
        
    # Fusionne les bits de la partie gauche et de la partie droite pour former la clé finale
    return (bit_slice(keyl, 39, 20) << 20) | bit_slice(keyr, 39, 20)

def lfsr_round(x):
    """
    Effectue une itération du registre à décalage à rétroaction linéaire (LFSR).
    
    Args:
    x (int): L'état actuel du LFSR.
    
    Returns:
    int: Le nouvel état du LFSR après une itération.
    """
    new_bit = bit(x, 0) ^ bit(x, 2) ^ bit(x, 19) ^ bit(x, 21)  # Calcule le nouveau bit
    new_state = (x >> 1) | (new_bit << 39)  # Décale et ajoute le nouveau bit
    return new_state

def dst80_round(keyl, keyr, s):
    """
    Effectue une itération d'une ronde dans l'algorithme DST-80.
    
    Args:
    keyl (int): Partie gauche de la clé.
    keyr (int): Partie droite de la clé.
    s (int): État interne.
    
    Returns:
    tuple: Un tuple contenant les nouvelles valeurs de keyl, keyr et s après la ronde.
    """
    k = dst80_merge(keyl, keyr)  # Fusionne les parties de clé
    s = (s >> 2) | ((f(k, s) ^ (s & 3)) << 38)  # Applique des opérations sur l'état interne s
    keyr = lfsr_round(keyr)  # Applique une itération LFSR à la partie droite de la clé
    keyl = lfsr_round(keyl)  # Applique une itération LFSR à la partie gauche de la clé
    return keyl, keyr, s

def dst80_rounds(keyl, keyr, s, nrounds):
    """
    Effectue plusieurs itérations de rondes dans l'algorithme DST-80.
    
    Args:
    keyl (int): Partie gauche de la clé.
    keyr (int): Partie droite de la clé.
    s (int): État interne.
    nrounds (int): Le nombre d'itérations de rondes à effectuer.
    
    Returns:
    int: L'état interne s après avoir effectué les itérations de rondes.
    """
    for i in range(nrounds):
        keyl, keyr, s = dst80_round(keyl, keyr, s)  # Applique une ronde
    return s


Finalement, il suffit de faire appel à la fonction dst80_rounds 200 fois (le nombre de rounds de DST80)

In [15]:
def dst80(keyl, keyr, challenge):
    """
    Applique l'algorithme DST-80 pour générer un résultat à partir des clés et du challenge.
    
    Args:
    keyl (int): Partie gauche de la clé.
    keyr (int): Partie droite de la clé.
    challenge (int): Challenge utilisé dans l'algorithme.
    
    Returns:
    int: Le résultat obtenu après avoir appliqué l'algorithme DST-80 et effectué un masquage.
    """
    result = dst80_rounds(keyl, keyr, challenge, 200) & 0xffffff  # Applique les itérations de rondes et effectue un masquage
    return result


Exemple d'exécution

In [23]:
# On prend par exemple des valeurs aléatoires pour KeyL et KeyR
keyL,keyR=0x955a3eaaaa ,0xaaaac1a56a
# On peut choisir aussi un challenge de 40bits quelconque 
Challenge= 0xC212345678

Il suffit ensuite d'appller dst80 avec ces arguments

In [22]:
Response= dst80(keyL, keyR, Challenge)
print(Response)

11046871
